In [1]:
# cv7ahand/annotations 폴더를 만들고
# 전체 이미지를 train, val, test 비율대로 나눠서
# (train_val_test_ratio를 설정하세요)
# cv7ahand_train.json, cv7ahand_val.json, cv7ahand_test.json 파일을 만듭니다
# json 파일들의 포맷은 freihand_val.json 파일의 포맷과 동일하게 맞춘겁니다

In [133]:
# 주피터 노트북에서 사용중인 가상환경 확인

import sys

if hasattr(sys, 'base_prefix'):
    print(f"base: {sys.base_prefix}")

if hasattr(sys, 'real_prefix'):
    print(f"real: {sys.real_prefix}")

if hasattr(sys, 'prefix'):
    print(f"prfx: {sys.prefix}")

base: /usr/local/Caskroom/miniconda/base/envs/xcda
prfx: /usr/local/Caskroom/miniconda/base/envs/xcda


In [134]:
# 이미지 리스트 불러오기

import os
from glob import glob

img_dir_path = "cv7ahand/training/rgb"
img_list = []
for path in glob(os.path.join(img_dir_path, '*jpg')):
    img_list.append(path)
    # print(path)

In [135]:
# shuffle 후 train, val, test divide

import random

train_val_test_ratio = [4, 3, 2]

random.shuffle(img_list)

num_train = len(img_list) * train_val_test_ratio[0] // sum(train_val_test_ratio)
num_val = len(img_list) * train_val_test_ratio[1] // sum(train_val_test_ratio)
num_test = len(img_list) - num_train - num_val

print(num_train, num_val, num_test)

img_list_train = img_list[:num_train]
img_list_val = img_list[num_train:num_train+num_val]
img_list_test = img_list[-num_test:]

print(len(img_list))
print(len(img_list_train), len(img_list_val), len(img_list_test))

563 422 282
1267
563 422 282


In [136]:
# freihand dataset의 "categories" 부분 확인

# import json

# with open('freihand_val.json') as f:
#     frei = json.load(f)

# frei['categories']

In [137]:
if not os.path.exists("cv7ahand/annotations"):
    os.mkdir("cv7ahand/annotations")

## cv7ahand_train.json

In [138]:
import json

d = {}

d['info'] = {
    'description': 'CV7AHand', 
    'version': '1.0', 
    'year': '2022', 
    'date_created': '2022-06-01'
}

d['licenses'] = ""

d['images'] = []

d['annotations'] = []

d['categories'] = [{
    'supercategory': 'hand',
    'id': 1,
    'name': 'hand',
    'keypoints': [
        'wrist',
        'thumb1',
        'thumb2',
        'thumb3',
        'thumb4',
        'forefinger1',
        'forefinger2',
        'forefinger3',
        'forefinger4',
        'middle_finger1',
        'middle_finger2',
        'middle_finger3',
        'middle_finger4',
        'ring_finger1',
        'ring_finger2',
        'ring_finger3',
        'ring_finger4',
        'pinky_finger1',
        'pinky_finger2',
        'pinky_finger3',
        'pinky_finger4'],
    'skeleton': [
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [1, 6],
        [6, 7],
        [7, 8],
        [8, 9],
        [1, 10],
        [10, 11],
        [11, 12],
        [12, 13],
        [1, 14],
        [14, 15],
        [15, 16],
        [16, 17],
        [1, 18],
        [18, 19],
        [19, 20],
        [20, 21]]}]

In [139]:
# d_json = json.dumps(d, indent = 4, ensure_ascii=False) 

# print(d_json)

In [140]:
from tqdm import tqdm
import cv2
import mediapipe as mp

with mp.solutions.hands.Hands(model_complexity=1) as hands:
    for img_path in tqdm(img_list_train):


        img_info = {}

        file_name = img_path[9:]
        # print(file_name)
        img_info['file_name'] = file_name

        img = cv2.imread(img_path)
        # print(img.shape)
        img_info['height'] = img.shape[0]
        img_info['width'] = img.shape[1]

        img_id = list(map(str, img_path.split('/')))[-1]
        img_id = img_id.replace(".jpg", "")
        img_id = int(img_id)
        # print(img_id)
        img_info['id'] = img_id

        d['images'].append(img_info)
        

        ann_info = {}

        ann_info['category_id'] = 1
        ann_info['id'] = img_id
        ann_info['image_id'] = img_id
        ann_info['iscrowd'] = 0

        ann_info['keypoint'] = []

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        kpts = hands.process(img)

        if kpts.multi_hand_landmarks:
            for hand_landmarks in kpts.multi_hand_landmarks:
                for i in range(21):
                    x, y = hand_landmarks.landmark[i].x, hand_landmarks.landmark[i].y
                    x, y = x * img.shape[0], y * img.shape[1]
                    # print(x, y)
                    ann_info['keypoint'].append(x)
                    ann_info['keypoint'].append(y)
                    ann_info['keypoint'].append(1)
                break

        # ann_info['bbox']

        # ann_info['segmentation']

        # ann_info['area']

        d['annotations'].append(ann_info)    


100%|██████████| 563/563 [00:21<00:00, 26.32it/s]


In [141]:
# d_json = json.dumps(d, indent = 4, ensure_ascii=False) 

# print(d_json)

In [142]:
with open("cv7ahand/annotations/cv7ahand_train.json", "w", encoding='utf-8') as f:
    json.dump(d, f, indent=4, ensure_ascii=False)

## cv7ahand_val.json

In [143]:
import json

d = {}

d['info'] = {
    'description': 'CV7AHand', 
    'version': '1.0', 
    'year': '2022', 
    'date_created': '2022-06-01'
}

d['licenses'] = ""

d['images'] = []

d['annotations'] = []

d['categories'] = [{
    'supercategory': 'hand',
    'id': 1,
    'name': 'hand',
    'keypoints': [
        'wrist',
        'thumb1',
        'thumb2',
        'thumb3',
        'thumb4',
        'forefinger1',
        'forefinger2',
        'forefinger3',
        'forefinger4',
        'middle_finger1',
        'middle_finger2',
        'middle_finger3',
        'middle_finger4',
        'ring_finger1',
        'ring_finger2',
        'ring_finger3',
        'ring_finger4',
        'pinky_finger1',
        'pinky_finger2',
        'pinky_finger3',
        'pinky_finger4'],
    'skeleton': [
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [1, 6],
        [6, 7],
        [7, 8],
        [8, 9],
        [1, 10],
        [10, 11],
        [11, 12],
        [12, 13],
        [1, 14],
        [14, 15],
        [15, 16],
        [16, 17],
        [1, 18],
        [18, 19],
        [19, 20],
        [20, 21]]}]

In [144]:
from tqdm import tqdm
import cv2
import mediapipe as mp

with mp.solutions.hands.Hands(model_complexity=1) as hands:
    for img_path in tqdm(img_list_val):


        img_info = {}

        file_name = img_path[9:]
        # print(file_name)
        img_info['file_name'] = file_name

        img = cv2.imread(img_path)
        # print(img.shape)
        img_info['height'] = img.shape[0]
        img_info['width'] = img.shape[1]

        img_id = list(map(str, img_path.split('/')))[-1]
        img_id = img_id.replace(".jpg", "")
        img_id = int(img_id)
        # print(img_id)
        img_info['id'] = img_id

        d['images'].append(img_info)
        

        ann_info = {}

        ann_info['category_id'] = 1
        ann_info['id'] = img_id
        ann_info['image_id'] = img_id
        ann_info['iscrowd'] = 0

        ann_info['keypoint'] = []

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        kpts = hands.process(img)

        if kpts.multi_hand_landmarks:
            for hand_landmarks in kpts.multi_hand_landmarks:
                for i in range(21):
                    x, y = hand_landmarks.landmark[i].x, hand_landmarks.landmark[i].y
                    x, y = x * img.shape[0], y * img.shape[1]
                    # print(x, y)
                    ann_info['keypoint'].append(x)
                    ann_info['keypoint'].append(y)
                    ann_info['keypoint'].append(1)
                break

        # ann_info['bbox']

        # ann_info['segmentation']

        # ann_info['area']

        d['annotations'].append(ann_info)    


100%|██████████| 422/422 [00:16<00:00, 25.17it/s]


In [145]:
with open("cv7ahand/annotations/cv7ahand_val.json", "w", encoding='utf-8') as f:
    json.dump(d, f, indent=4, ensure_ascii=False)

## cv7ahand_test.json

In [146]:
import json

d = {}

d['info'] = {
    'description': 'CV7AHand', 
    'version': '1.0', 
    'year': '2022', 
    'date_created': '2022-06-01'
}

d['licenses'] = ""

d['images'] = []

d['annotations'] = []

d['categories'] = [{
    'supercategory': 'hand',
    'id': 1,
    'name': 'hand',
    'keypoints': [
        'wrist',
        'thumb1',
        'thumb2',
        'thumb3',
        'thumb4',
        'forefinger1',
        'forefinger2',
        'forefinger3',
        'forefinger4',
        'middle_finger1',
        'middle_finger2',
        'middle_finger3',
        'middle_finger4',
        'ring_finger1',
        'ring_finger2',
        'ring_finger3',
        'ring_finger4',
        'pinky_finger1',
        'pinky_finger2',
        'pinky_finger3',
        'pinky_finger4'],
    'skeleton': [
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [1, 6],
        [6, 7],
        [7, 8],
        [8, 9],
        [1, 10],
        [10, 11],
        [11, 12],
        [12, 13],
        [1, 14],
        [14, 15],
        [15, 16],
        [16, 17],
        [1, 18],
        [18, 19],
        [19, 20],
        [20, 21]]}]

In [147]:
from tqdm import tqdm
import cv2
import mediapipe as mp

with mp.solutions.hands.Hands(model_complexity=1) as hands:
    for img_path in tqdm(img_list_test):


        img_info = {}

        file_name = img_path[9:]
        # print(file_name)
        img_info['file_name'] = file_name

        img = cv2.imread(img_path)
        # print(img.shape)
        img_info['height'] = img.shape[0]
        img_info['width'] = img.shape[1]

        img_id = list(map(str, img_path.split('/')))[-1]
        img_id = img_id.replace(".jpg", "")
        img_id = int(img_id)
        # print(img_id)
        img_info['id'] = img_id

        d['images'].append(img_info)
        

        ann_info = {}

        ann_info['category_id'] = 1
        ann_info['id'] = img_id
        ann_info['image_id'] = img_id
        ann_info['iscrowd'] = 0

        ann_info['keypoint'] = []

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        kpts = hands.process(img)

        if kpts.multi_hand_landmarks:
            for hand_landmarks in kpts.multi_hand_landmarks:
                for i in range(21):
                    x, y = hand_landmarks.landmark[i].x, hand_landmarks.landmark[i].y
                    x, y = x * img.shape[0], y * img.shape[1]
                    # print(x, y)
                    ann_info['keypoint'].append(x)
                    ann_info['keypoint'].append(y)
                    ann_info['keypoint'].append(1)
                break

        # ann_info['bbox']

        # ann_info['segmentation']

        # ann_info['area']

        d['annotations'].append(ann_info)    


100%|██████████| 282/282 [00:12<00:00, 23.36it/s]


In [148]:
with open("cv7ahand/annotations/cv7ahand_test.json", "w", encoding='utf-8') as f:
    json.dump(d, f, indent=4, ensure_ascii=False)